# Introdução

## Importações

In [1]:
import glob
import os

import polars as pl

## Variáveis globais

In [2]:
DATASET_DIRECTORY = "./CICIoT2023/"

## Funções Auxiliares

In [3]:
def sample_rows(
    group_by: pl.dataframe.group_by.GroupBy, percentage: float
) -> pl.DataFrame:
    dfs = []
    for _, df in group_by:
        if len(df) * percentage <= 1:
            dfs.append(df.sample(n=1))
            continue
        dfs.append(df.sample(fraction=percentage))

    return pl.concat(dfs)


def generate_dataframe(file_list: list, percentage: float) -> pl.DataFrame:
    dfs = []
    for file in file_list:
        df = pl.read_csv(file)
        dfs.append(
            sample_rows(
                group_by=df.group_by(["label"], maintain_order=True),
                percentage=percentage,
            )
        )
    return pl.concat(dfs)

# Criando um dataset menor

In [4]:
file_dir = "./data/"
if not os.path.exists(file_dir):
    os.makedirs(file_dir)

df_sets = sorted(glob.glob(os.path.join(DATASET_DIRECTORY, "*.csv")))

In [5]:
generate_dataframe(file_list=df_sets, percentage=0.001).write_csv(
    file="./data/dataset_dev.csv"
)

In [6]:
generate_dataframe(file_list=df_sets, percentage=0.01).write_csv(
    file="./data/dataset_one_percent.csv"
)

In [7]:
generate_dataframe(file_list=df_sets, percentage=0.1).write_csv(
    file="./data/dataset_ten_percent.csv"
)